抓取百度失信人名单
创建爬虫项目
scrapy startproject dishonest
3120305689_段克强

实现步骤
- 创建爬虫项目	
- 根据需求, 定义数据模型
- 实现百度失信名单爬虫
- 保存失信名单信息
- 实现随机User-Agent和代理IP下载器中间件, 解决IP反爬

In [1]:
# scrapy框架新建item.py定义数据模型
# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://doc.scrapy.org/en/latest/topics/items.html

"""
3.3. 根据需求, 定义数据数据模型
步骤:
1. 定义数据模型类:DishonestItem, 继承scrapy.Item
2. 定义要抓取字段
    失信人名称
    失信人号码
    失信人年龄
    区域
    法人(企业)
    失信内容
    公布日期
    公布/执行单位
    创建日期
    更新日期
"""

import scrapy

class DishonestItem(scrapy.Item):
    # define the fields for your item here like:
    # 失信人名称
    name = scrapy.Field()
    # 失信人证件号
    card_num = scrapy.Field()
    # 失信人年龄, 企业年龄都是0
    age = scrapy.Field()
    # 区域
    area = scrapy.Field()
    # 法人(企业)
    business_entity = scrapy.Field()
    # 失信内容
    content = scrapy.Field()
    # 公布日期
    publish_date = scrapy.Field()
    # 公布/执行单位
    publish_unit = scrapy.Field()
    # 创建日期
    create_date = scrapy.Field()
    # 更新日期
    update_date = scrapy.Field()


分析“百度失信人”网站确定URL列表
实现爬虫


In [2]:
import requests
import mjson

url = 'https://sp0.baidu.com/8aQDcjqpAAV3otqbppnN2DJv/api.php?resource_id=6899&query=%E5%A4%B1%E4%BF%A1%E4%BA%BA&pn=10&rn=10&from_mid=1&ie=utf-8&oe=utf-8'

# 代码尝试是需要'User-Agent'与'Referer'才能发送请求
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'Referer':'https://www.baidu.com/s?ie=utf-8&f=8&rsv_bp=1&rsv_idx=1&tn=baidu&wd=%E5%A4%B1%E4%BF%A1%E4%BA%BA&fenlei=256&oq=%25E4%25B8%25AD%25E5%259B%25BD%25E6%25B3%2595%25E9%2599%25A2%25E5%2586%25B3%25E8%2583%259C%25E5%259F%25BA%25E6%259C%25AC%25E8%25A7%25A3%25E5%2586%25B3%25E6%2589%25A7%25E8%25A1%258C%25E9%259A%25BE%25E4%25BF%25A1%25E6%2581%25AF%25E7%25BD%2591&rsv_pq=d0684c9b00034cdb&rsv_t=6c7d4xNKEKH01bAc6AR%2Fr98UN1NWFQ7zWW1VpzlddOziC8KByQZG1Gy8wKw&rqlang=cn&rsv_enter=1&rsv_dl=tb&rsv_btype=t&inputT=1772&rsv_sug3=92&rsv_sug1=37&rsv_sug7=100&rsv_sug2=0&rsv_sug4=2210&rsv_sug=1'
}

response = requests.get(url, headers=headers)


print(response.content.decode())  

{"status":"0","t":"","set_cache_time":"","data":[{"version":2,"status":1,"resNum":10,"dispNum":30000,"listNum":2000,"disp_data":[{"StdStg":6899,"StdStl":8,"_update_time":"1611152896","cambrian_appid":"0","changefreq":"always","age":"52","areaName":"广东","areaNameNew":"广东","businessEntity":"","cardNum":"4418811963****1111","caseCode":"(2005)清英法执字第00799号","courtName":"英德市人民法院","disruptTypeName":"其他有履行能力而拒不履行生效法律文书确定义务","duty":"支付赔偿费2628元","focusNumber":"0","gistId":"(2005)清中法民一终字373号","gistUnit":"广东省清远市中级人民法院","iname":"陈国强","partyTypeName":"0","performance":"全部未履行","performedPart":"","publishDate":"2015年04月28日","publishDateStamp":"1430150400","regDate":"20051130","sexy":"男","sitelink":"http:\/\/zxgk.court.gov.cn\/","title":"失信人查询","type":"失信被执行人名单","unperformPart":"","lastmod":"2021-01-20T17:21:55","loc":"http:\/\/shixin.court.gov.cn\/detail?id=97073256","priority":"1.0","SiteId":2015330,"_version":1334,"_select_time":1611150979,"xzhId":"0"},{"StdStg":6899,"StdStl":8,"_update_time":"16111

在spiders文件夹下新建baidu.py完成爬虫的额具体实现
步骤：
- 起始URL
- 生成所有页面的请求
- 解析页面，提取需要的数据


In [4]:
from datetime import datetime
import scrapy
import json
from jsonpath import jsonpath
# from dishonest.items import DishonestItem
"""
完善爬虫
步骤
    1.起始URL
    2.生成所有页面的请求
    3.解析页面，提取需要的数据
"""

class BaiduSpider(scrapy.Spider):
    name = 'baidu'
    allowed_domains = ['baidu.com']
    start_urls = ['https://sp0.baidu.com/8aQDcjqpAAV3otqbppnN2DJv/api.php?resource_id=6899&query=%E5%A4%B1%E4%BF%A1%E4%BA%BA&pn=0&rn=10&from_mid=1&ie=utf-8&oe=utf-8']

    def parse(self, response):
        # 构建所有页面的请求
        # 把响应的json字符串转为字典
        results = json.loads(response.text)
        # 取出总数据条数
        disp_num = jsonpath(results, '$..dispNum')[0]
        # print(disp_num)
        # URL模板
        url_pattern = 'https://sp0.baidu.com/8aQDcjqpAAV3otqbppnN2DJv/api.php?resource_id=6899&query=%E5%A4%B1%E4%BF%A1%E4%BA%BA&pn={}&rn=10&from_mid=1&ie=utf-8&oe=utf-8'
        # 每隔10条数据，构建一个请求
        for pn in range(0, disp_num , 10):
            # 构建URL
            url = url_pattern.format(pn)
            # 创建请求，交给引擎
            yield scrapy.Request(url, callback=self.parse_data)

    def parse_data(self, response):
        """解析数据"""
        # 响应数据
        datas = json.loads(response.text)
        results = jsonpath(datas, '$..disp_data')[0]
        # 遍历结果列表
        for result in results:
            item = DishonestItem()
            # 失信人名称
            item['name'] = result['iname']
            # 失信人号码
            item['card_num'] = result['cardNum']
            # 失信人年龄
            item['age'] = int(result['age'])
            # 区域
            item['area'] = result['areaName']
            # 法人(企业)
            item['business_entity'] = result['businessEntity']
            # 失信内容
            item['content'] = result['duty']
            # 公布日期
            item['publish_date'] = result['publishDate']
            # 公布/执行单位
            item['publish_unit'] = result['courtName']
            # 创建日期
            item['create_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            # 更新日期
            item['update_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            # print(item)
            # 把数据交给引擎
            yield item



完成中间件操作
- UA伪装
- IP反爬

In [6]:
# Define here the models for your spider middleware
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/spider-middleware.html

from scrapy import signals

# useful for handling different item types with a single interface
from itemadapter import is_item, ItemAdapter
"""
实现随机User-Agent下载器中间
    1. 准备User-Agent列表
    2. 定义RandomUserAgent类
    3. 实现process_request方法, 设置随机的User-Agent
"""
import random
import requests
from redis import StrictRedis
import pickle

# from dishonest.settings import USER_AGENTS





class RandomUserAgent(object):

    def process_request(self, request, spider):

        #  3. 实现process_request方法, 设置随机的User-Agent
        request.headers['User-Agent'] = random.choice(USER_AGENTS)

        return None

"""
实现代理IP下载器中间件
    1. 定义ProxyMiddleware类
    2. 实现process_request方法, 设置代理IP
"""

class ProxyMiddleware(object):

    def process_request(self, request, spider):
        # 实现process_request方法, 设置代理IP


        # 1. 获取协议头
        protocol = request.url.split('://')[0]
        # 2. 构建代理IP请求的URL
        proxy_url = 'http://localhost:16888/random?protocol={}'.format(protocol)
        # 3. 发送请求, 获取代理IP
        response = requests.get(proxy_url)
        # 4. 把代理IP设置给request.meta['proxy']
        request.meta['proxy'] = response.content.decode()

        return None



建立setting.py完善相关配置信息


In [7]:
# Scrapy settings for dishonest project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://docs.scrapy.org/en/latest/topics/settings.html
#     https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://docs.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'dishonest'

SPIDER_MODULES = ['dishonest.spiders']
NEWSPIDER_MODULE = 'dishonest.spiders'


# Crawl responsibly by identifying yourself (and your website) on the user-agent
#USER_AGENT = 'dishonest (+http://www.yourdomain.com)'

# Obey robots.txt rules
# 设置为false 免除机器人协议
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32

# Configure a delay for requests for the same website (default: 0)
# See https://docs.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
#DOWNLOAD_DELAY = 3
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
#COOKIES_ENABLED = False

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
DEFAULT_REQUEST_HEADERS = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'Referer':'https://www.baidu.com/s?ie=utf-8&f=8&rsv_bp=1&rsv_idx=1&tn=baidu&wd=%E5%A4%B1%E4%BF%A1%E4%BA%BA&fenlei=256&oq=%25E4%25B8%25AD%25E5%259B%25BD%25E6%25B3%2595%25E9%2599%25A2%25E5%2586%25B3%25E8%2583%259C%25E5%259F%25BA%25E6%259C%25AC%25E8%25A7%25A3%25E5%2586%25B3%25E6%2589%25A7%25E8%25A1%258C%25E9%259A%25BE%25E4%25BF%25A1%25E6%2581%25AF%25E7%25BD%2591&rsv_pq=d0684c9b00034cdb&rsv_t=6c7d4xNKEKH01bAc6AR%2Fr98UN1NWFQ7zWW1VpzlddOziC8KByQZG1Gy8wKw&rqlang=cn&rsv_enter=1&rsv_dl=tb&rsv_btype=t&inputT=1772&rsv_sug3=92&rsv_sug1=37&rsv_sug7=100&rsv_sug2=0&rsv_sug4=2210&rsv_sug=1'
}

# Enable or disable spider middlewares
# See https://docs.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'dishonest.middlewares.DishonestSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
DOWNLOADER_MIDDLEWARES = {
   'dishonest.middlewares.RandomUserAgent': 543,
}

# Enable or disable extensions
# See https://docs.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
   'dishonest.pipelines.DishonestPipeline': 300,
}

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/autothrottle.html
#AUTOTHROTTLE_ENABLED = True
# The initial download delay
#AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
#AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'

# 配置MYSQL
# MYSQL的主机IP地址
MYSQL_HOST = 'localhost'
# MYSQL端口号
MYSQL_PORT = 3306
# MYSQL用户名
MYSQL_USER = 'root'
# MYSQL密码
MYSQL_PASSWORD = 'root'
# MYSQL数据库名
MYSQL_DB = 'dishonest'


#   1. 准备User-Agent列表
# 准备请求头
USER_AGENTS = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
    "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
    "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10"
]

建立pipelines.py对数据进行去重存储在Mysql数据库中


In [9]:
# -*- coding: utf-8 -*-
import pymysql
# from dishonest.settings import MYSQL_HOST, MYSQL_PORT, MYSQL_DB, MYSQL_USER, MYSQL_PASSWORD

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://doc.scrapy.org/en/latest/topics/item-pipeline.html
"""
实现管道类

步骤:
1. 在open_spider中, 建立数据库连接, 获取操作的数据的cursor
2. 在close_spider中, 关闭cursor,关闭数据库连接
3. 在process_item中,如果数据不存, 保存数据;
"""
class DishonestPipeline(object):

    def open_spider(self, spider):
        """
        1. open_spider中, 建立数据库连接, 获取操作的数据的cursor
        """
        # 建立数据库连接
        self.connection = pymysql.connect(host=MYSQL_HOST, port=MYSQL_PORT, db=MYSQL_DB,
                                          user=MYSQL_USER, password=MYSQL_PASSWORD)
        # 获取操作数据的cursor
        self.cursor = self.connection.cursor()

    def close_spider(self, spider):
        """
        2. 在close_spider中, 关闭cursor,关闭数据库连接
        """
        # 1. 关闭cursor
        self.cursor.close()
        # 2. 关闭数据库连接
        self.connection.close()


    def process_item(self, item, spider):
        """
        3. 在process_item中, 如果数据不存, 保存数据;
        """
        # 如果是自然人, 根据证件号进行判断
        # 如果是企业/组织: 企业名称 和 区域进行判断
        # 如何判断是企业还是自然人? 如果年龄为0就是企业, 否则就是自然人
        if item['age'] == 0:
            # 如果是企业, 根据 企业名称 和 区域进行判断 是否重复
            select_count_sql = "SELECT COUNT(1) from dishonest WHERE name = '{}' and area = '{}'".\
                format(item['name'], item['area'])
        else:
            card_num = item['card_num']
            # 如果证件号是18位, 那么就倒数第7到倒数第4位(不包含), 三个数字使用了**** 替换掉
            if len(card_num) == 18:
                card_num = card_num[:-7] + '****' + card_num[-4:]
                # print(card_num)
                # print(len(card_num))
                # 为了保护失信人隐私 和 数据格式一致性, 把修改后的数据赋值回去
                item['card_num'] = card_num

            # 否则就是自然人
            select_count_sql = "SELECT COUNT(1) from dishonest WHERE card_num = '{}'".format(item['card_num'])

        # 执行查询SQL
        self.cursor.execute(select_count_sql)
        # 获取查询结果
        count = self.cursor.fetchone()[0]

        if count == 0:
            keys, values = zip(*dict(item).items())
            # 如果没有数据, 就插入数据
            insert_sql = 'INSERT INTO dishonest ({}) VALUES ({})'.format(
                ','.join(keys),
                ','.join(['%s'] * len(values))
            )
            # 执行SQL
            self.cursor.execute(insert_sql, values)
            # 提交事务
            self.connection.commit()
            spider.logger.info('插入数据')
        else:
            # 否则就重复了
            spider.logger.info('数据重复')

        return item


# 运行项目需要在pycharm中建立scrapy项目生成结果!!!!
# 相应模块代码填充完毕后
# 命令行键入scrpy crawl baidu运行项目


## 将数据爬取到MySQL数据库后，运用pyecharts包完成数据可视化操作
## 在jupyter的根目录生成html文件点开即可观看可视化结果

In [10]:
from pyecharts.charts import Bar,Pie   # 注意，新版本中，Pie被放进了charts
from pyecharts import options as opts
# from job_pac.utils_mysql import Db_MySQL
import pandas as pd
import pymysql

# 配置MYSQL
# MYSQL的主机IP地址
MYSQL_HOST = 'localhost'
# MYSQL端口号
MYSQL_PORT = 3306
# MYSQL用户名
MYSQL_USER = 'root'
# MYSQL密码
MYSQL_PASSWORD = 'root'
# MYSQL数据库名
MYSQL_DB = 'dishonest'


# 建立数据库连接
connection = pymysql.connect(host=MYSQL_HOST, port=MYSQL_PORT, db=MYSQL_DB, user=MYSQL_USER, password=MYSQL_PASSWORD)

# 创建游标
cursor = connection.cursor()

# sql语句
sql = "select area as '地域', count(dishonest_id) as '数量' from dishonest GROUP BY area"
# sql = "select distinct area as '地域' from dishonest "


# 执行查询SQL
cursor.execute(sql);
# 获取查询结果
result = cursor.fetchall()

# 4.存储结果-多组数据单独存放
namelist = []
numlist = []
for name in result:
    namelist.append(name[0])
    numlist.append(name[1])
print(namelist)
print(numlist)

# 4.存储结果-多组数据存在一起。
lists = []
for item in result:
    for i in range(0, len(item)):
        # print('查询到的数据为：', item, type(item), len(item))  # 是数组tuple，
        lists.append(item[i])
print('存储：', lists)

## 数据可视化
from pyecharts.globals import ThemeType         # 内置主题类型

# 按地域绘制饼图
pie = Pie()
pie.add("占比",
        [list(z) for z in zip(namelist,numlist)],
        center=["30%", "50%"],
        )
pie.set_global_opts(
    title_opts=opts.TitleOpts(title="失信人地域分布图"),
    legend_opts=opts.LegendOpts(pos_left="35%"),
)
# 设置显示的样子，加入了百分比
pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c} ({d}%)"))
pie.render('失信人地域分布图.html')

# df_result = pd.DataFrame(list(result), columns=["area", "count"])
# print(df_result.shape)
# print(df_result.head())

# 1.关闭cursor
cursor.close()
# 2. 关闭数据库连接
connection.close()

['上海', '云南', '内蒙古', '北京', '四川', '天津', '宁夏', '安徽', '山东', '山西', '广东', '广西', '新疆', '江苏', '江西', '河北', '河南', '浙江', '海南', '湖北', '湖南', '甘肃', '福建', '贵州', '辽宁', '重庆', '陕西', '青海', '黑龙江']
[5, 5, 12, 5, 12, 5, 6, 38, 20, 8, 53, 12, 3, 68, 12, 25, 55, 117, 1, 14, 5, 10, 33, 25, 13, 13, 1, 1, 6]
存储： ['上海', 5, '云南', 5, '内蒙古', 12, '北京', 5, '四川', 12, '天津', 5, '宁夏', 6, '安徽', 38, '山东', 20, '山西', 8, '广东', 53, '广西', 12, '新疆', 3, '江苏', 68, '江西', 12, '河北', 25, '河南', 55, '浙江', 117, '海南', 1, '湖北', 14, '湖南', 5, '甘肃', 10, '福建', 33, '贵州', 25, '辽宁', 13, '重庆', 13, '陕西', 1, '青海', 1, '黑龙江', 6]


In [11]:
from pyecharts.charts import Bar,Pie   # 注意，新版本中，Pie被放进了charts
from pyecharts import options as opts
import pymysql


# 配置MYSQL
# MYSQL的主机IP地址
MYSQL_HOST = 'localhost'
# MYSQL端口号
MYSQL_PORT = 3306
# MYSQL用户名
MYSQL_USER = 'root'
# MYSQL密码
MYSQL_PASSWORD = 'root'
# MYSQL数据库名
MYSQL_DB = 'dishonest'

# 建立数据库连接
connection = pymysql.connect(host=MYSQL_HOST, port=MYSQL_PORT, db=MYSQL_DB, user=MYSQL_USER, password=MYSQL_PASSWORD)

# 创建游标
cursor = connection.cursor()

# sql语句
sql = "select age as '年龄', count(dishonest_id) as '数量' from dishonest GROUP BY age"
# sql = "select distinct area as '地域' from dishonest "


# 执行查询SQL
cursor.execute(sql);
# 获取查询结果
result = cursor.fetchall()

# 4.存储结果-多组数据单独存放
namelist = []
numlist = []
for name in result:
    namelist.append(name[0])
    numlist.append(name[1])
print(namelist)
print(numlist)

# 4.存储结果-多组数据单独存放
namelist = []
numlist = []
for name in result:
    namelist.append(name[0])
    numlist.append(name[1])
print(namelist)
print(numlist)


# 数据可视化
from pyecharts.globals import ThemeType         # 内置主题类型

# 柱状图
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.MACARONS))
# x为发布日期，y为客运量 间隔
bar.add_xaxis(namelist)
bar.add_yaxis("人数", numlist, category_gap="50%")
# 基本设置
bar.set_global_opts(title_opts=opts.TitleOpts(title="失信人年龄分布图"),
                    yaxis_opts=opts.AxisOpts(name="人数(单位:人)"),  # 设置y轴名字，x轴同理
                    xaxis_opts=opts.AxisOpts(name="年龄"),
                    datazoom_opts=opts.DataZoomOpts()  # 设置水平缩放，默认可滑动(*好东西)
                    )
# 标记峰值
bar.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True),
    markpoint_opts=opts.MarkPointOpts(
        data=[
            opts.MarkPointItem(type_='max', name='最大值'),  # 最大值标记点
            opts.MarkPointItem(type_='min', name='最小值'),  # 最小值标记点
            opts.MarkPointItem(type_='average', name='平均值')  # 平均值标记点
        ]
    )
)

bar.render('失信人年龄分布图.html')  # 输出为html

# 1.关闭cursor
cursor.close()
# 2. 关闭数据库连接
connection.close()

[0, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 75, 76]
[74, 1, 2, 2, 3, 3, 5, 3, 8, 10, 7, 19, 12, 7, 19, 11, 12, 14, 17, 15, 20, 21, 18, 16, 13, 20, 22, 17, 15, 20, 13, 19, 22, 20, 10, 11, 8, 12, 4, 6, 4, 3, 2, 2, 5, 2, 1, 2, 4, 1, 2, 1, 1, 1, 1]
[0, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 75, 76]
[74, 1, 2, 2, 3, 3, 5, 3, 8, 10, 7, 19, 12, 7, 19, 11, 12, 14, 17, 15, 20, 21, 18, 16, 13, 20, 22, 17, 15, 20, 13, 19, 22, 20, 10, 11, 8, 12, 4, 6, 4, 3, 2, 2, 5, 2, 1, 2, 4, 1, 2, 1, 1, 1, 1]
